In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn import preprocessing

In [2]:
df = pd.read_excel('./files/서울 지하철역 마스터시트_0811.xlsx')
df = df.fillna(0)
df.describe()

,역위도,역경도,근처 공연장수,공연건수,예매건수,퇴근시간 하차인원(17시~1시),신용판매금액,문화예술관람,한 공연당 예매건수
count,234.000000,234.000000,234.000000,234.000000,2.340000e+02,2.340000e+02,234.000000,234.000000,234.000000
mean,37.546635,127.006856,29.286325,262.871795,4.329116e+05,2.855032e+06,1566.431624,5.758974,54114.946032
std,0.045858,0.083001,44.585541,380.784384,7.243934e+05,2.486570e+06,345.654591,2.252558,92803.960890
min,37.464250,126.801500,0.000000,0.000000,0.000000e+00,0.000000e+00,1092.000000,1.600000,0.000000
25%,37.510303,126.945800,4.000000,25.500000,9.640750e+03,1.284361e+06,1320.000000,4.100000,2451.713000
50%,37.546250,127.015950,12.000000,92.000000,1.394010e+05,2.090592e+06,1467.000000,5.300000,14962.430000
75%,37.572080,127.069525,30.000000,317.500000,4.051542e+05,3.296192e+06,1631.250000,7.400000,47092.552500
max,37.689230,127.165300,167.000000,1464.000000,2.845374e+06,1.665208e+07,2336.000000,10.800000,365279.200000


In [38]:
def get_score(level, data):
    '''
    Description :
    level안에 있는 원소를 기준으로
    1 ~ len(level)+ 1 까지 점수를 부여하는 함수
    
    Parameters :
    level = 튜플 또는 리스트 타입의 숫자형 데이터이며 반드시 오름차순으로 정렬되어 있어야함.
    예 - [1,2,3,4,5] O, [5,4,3,2,1] X, [1,3,2,10,4] X 
    data = 점수를 부여할 데이터. 순회가능한(iterable) 데이터 형식
    return :
    점수를 담고 있는 리스트 반환
    '''
    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue 
    return score 

In [48]:
def get_grade(df, num_class, tick_point, col_map):
    '''
    Description :
    각 데이터에 대한 열의 변수값이 주어졌을때
    변수별로 점수를 계산하여 주어진 데이터 오른쪽에 붙여줍니다.
    
    Parameters :
    df = pandas.DataFrame 데이터
    num_class = 등급(점수) 개수
    tick_point = 각 점수에 대해서 등급을 나눌 기준이 되는 값
                    'quantile', 'min_max' 또는 리스트를 통하여 직접 값을 정할 수 있음.
                    단, 리스트 사용시 원소의 개수는 반드시 num_class - 1 이어야함.
                    quatile = 데이터의 분위수를 기준으로 점수를 매김
                    min_max = 데이터의 최소값과 최대값을 동일 간격으로 나누어 점수를 매김
    col_map = 각 점수에 해당하는 칼럼명
    예 - {'score_1':'column_1','score_2':'column_2','score_3':'column_3'}
    
    Return : 
    pandas.DataFrame
    '''

    ##### 필요모듈 체크
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing

    ##### 파라미터 체크
    if not isinstance(df, pd.DataFrame): ## 데이터는 pd.DataFrame이어야 함.
        print('데이터는 pandas.DataFrame 객체여야 합니다.')
        return

    if isinstance(tick_point, dict) == False or isinstance(col_map, dict) == False: ## tick_point와 col_map은 모두 딕셔너리
        print(f'tick_point와 col_map은 모두 딕셔너리여야합니다.')
        return

    if not set(col_map.values()) == set(df[col_map.values()].columns):
        print('데이터 칼럼을 다시 확인해주세요')

    for k, v in tick_point.items():
        if isinstance(v, str):
            if not v in ['quantile','min_max']:
                print(f'{k}의 값은 "quantile" 또는 "min_max"중에 하나여야 합니다.')
                return
        elif isinstance(v,list) or isinstance(v,tuple):
            if len(v) != num_class[k]-1:
                print(f'{k}에 대응하는 리스트(튜플)의 원소는 {num_class[k]-1}개여야 합니다.')
                return


    for k, v in tick_point.items():
    # 데이터 표준화
        scale = preprocessing.StandardScaler() ## 데이터의 범위 조작하기 쉽게 해주는 클래스 
        temp_data = np.array(df[col_map[k]]) ## 데이터를 Numpy 배열로 변환
        temp_data = temp_data.reshape((-1,1)) ## scale을 적용하기위해 1차원 배열을 2차원으로 변환
        temp_data = scale.fit_transform(temp_data) ## 데이터를 평균은 0, 표준편차는 1을 갖도록 변환 
        temp_data = temp_data.squeeze() ## 데이터를 다시 1차원으로 변환

        if isinstance(v,str):
            if v == 'quantile':    
                ## 분위수 벡터
                quantiles_level = np.linspace(0,1,num_class[k]+1)[1:-1] ## 분위수를 구할 기준값을 지정 0과 1은 제외
                quantiles = [] ## 분위수를 담을 리스트
                for ql in quantiles_level:
                    quantiles.append(np.quantile(temp_data,ql)) ## 분위수를 계산하고 리스트에 삽입

            else: ## min_max인 경우      
                ## 등분점 계산
                quantiles = np.linspace(np.min(temp_data),np.max(temp_data),num_class[k]+1)[1:-1] ## 최소값과 최대값을 점수 개수만큼 등간격으로 분할하는 점

        else: ## 직접 구분값을 넣어주는 경우
            quantiles = v ## 직접 구분값을 넣어줌
        score = get_score(quantiles, temp_data) ## 구분값을 기준으로 점수를 부여하고 리스트로 저장한다.
        new_col_name = col_map[k]+'_'+k ## 점수값을 담는 변수의 이름

        df[new_col_name] = score ## 기존데이터 옆에 점수 데이터를 추가한다.
    return df

In [51]:
tick_point={'score_1':'quantile','score_2':'quantile',
            'score_3':'quantile', 'score_4':'quantile'}
col_map={'score_1':'퇴근시간 하차인원(17시~1시)','score_2':'한 공연당 예매건수','score_3':'신용판매금액',
        'score_4':'문화예술관람'}
num_class= {'score_1':5,'score_2':5,'score_3':3,'score_4':3}

In [57]:
result = get_grade(df=df, num_class=num_class, tick_point = tick_point, col_map = col_map)
result[['퇴근시간 하차인원(17시~1시)_score_1','한 공연당 예매건수_score_2',
        '신용판매금액_score_3','문화예술관람_score_4']]

,퇴근시간 하차인원(17시~1시)_score_1,한 공연당 예매건수_score_2,신용판매금액_score_3,문화예술관람_score_4
0,4,2,3,1
1,3,2,1,3
2,5,4,3,3
3,1,4,3,3
4,4,2,2,1
...,...,...,...,...
229,4,1,1,3
230,5,1,2,2
231,3,2,2,2
232,1,5,1,1


In [69]:
result['문화점수'] = (result['한 공연당 예매건수_score_2']+result['문화예술관람_score_4'])/2              
result['인구점수'] = (result['퇴근시간 하차인원(17시~1시)_score_1']+result['신용판매금액_score_3'])/2              
result['최종점수'] = (result['문화점수']+ result['인구점수'])/2
final_df = result[['역사명','자치구','문화점수','인구점수','최종점수']].sort_values(by='최종점수', ascending=False)
final_df = final_df.reset_index(drop=True)
final_df.head(50)   #4점만점

,역사명,자치구,문화점수,인구점수,최종점수
0,강남,강남구,3.5,4.0,3.75
1,신사,강남구,4.0,3.5,3.75
2,목동,양천구,3.5,4.0,3.75
3,오목교,양천구,3.5,4.0,3.75
4,압구정,강남구,4.0,3.5,3.75
5,역삼,강남구,3.5,3.5,3.50
6,왕십리,성동구,4.0,3.0,3.50
7,구의,광진구,3.5,3.5,3.50
8,성신여대입구,성북구,4.0,3.0,3.50
9,구파발,은평구,3.5,3.5,3.50


In [62]:
final_df.value_counts('자치구')

자치구
강남구    12
광진구     7
마포구     7
성동구     7
서초구     4
양천구     4
성북구     3
송파구     2
은평구     2
용산구     1
중구      1
dtype: int64